### Практикум по программированию
### ПИ19-1в
### Заваруев Иван
#### Лабораторная 2. Сложная анимация. 03.11.2020
#### Муравей Лэнгтона

In [9]:
from tkinter import *
from itertools import cycle
from time import sleep


SIZE = 10 # Размер поля
TILES = 50 # Размер одной клетки


# Цвета поля
COLORS = {
    'ant': 'black',
    1: 'lightblue',
    2: 'orange',
}


# Каталог всех возможных движений муравья
MOVES = {
    'N': {
        'L' : {
            'shift': (0, -1),
            'head': 'W',
        },
        'R' : {
            'shift': (0, 1),
            'head': 'E',
        },
    },
    'E': {
        'L' : {
            'shift': (-1, 0),
            'head': 'N',
        },
        'R' : {
            'shift': (1, 0),
            'head': 'S',
        },
    },
    'S': {
        'L' : {
            'shift': (0, 1),
            'head': 'E',
        },
        'R' : {
            'shift': (0, -1),
            'head': 'W',
        },
    },
    'W': {
        'L' : {
            'shift': (-1, 0),
            'head': 'S',
        },
        'R' : {
            'shift': (1, 0),
            'head': 'N',
        },
    },
}


# Создание приложения
root = Tk()
root.title('Муравей')
root.geometry(f'{TILES*SIZE}x{TILES*SIZE+50}')


# Создание канвы
c = Canvas(root,
          width=TILES*SIZE,
          height=TILES*SIZE,
          )
c.pack()


# Класс клетки
class Tile():
    def __init__(self, c, x, y, size, ):
        self.c = c
        self.color = COLORS[1]
        self.id = self.c.create_rectangle(x, # Создается объект rectangle канвы
                                     y,
                                     x+size,
                                     y+size,
                                     fill=self.color,
                                     activestipple='gray50',
                                    )
    
    #Сброс к начальному цвету
    def reset(self): 
        self.color = COLORS[1]
        self.c.itemconfigure(self.id, fill=COLORS[1])
    
    
    # Переключение на другой цвет
    def switch(self):
        if self.color == COLORS[1]:
            self.color = COLORS[2]
            self.c.itemconfigure(self.id, fill=COLORS[2])            
        elif self.color == COLORS[2]:
            self.color = COLORS[1]
            self.c.itemconfigure(self.id, fill=COLORS[1])
            
    
    # Получение статуса клетки с муравьем
    def arrived(self):
        self.c.itemconfigure(self.id, fill=COLORS['ant'])
        
    
    # Обработка покидания муравьем клетки
    def left(self):
        if self.color == COLORS[2]:
            clr = COLORS[1]
        else: 
            clr = COLORS[2]
        self.color = clr
        self.c.itemconfigure(self.id, fill=clr)
    

# Хендлер левой кнопки мыши - смены цвета клетки
def __switchTile(event):
    x = c.canvasx(event.x)
    y = c.canvasy(event.y)
    tile_num = c.find_closest(x,y)[0]
    tile = tiles[(tile_num-1)//TILES][tile_num%TILES-1]
    if tile_num == ant.id:
        return    
    tile.switch()
        

# Хендлер правой кнопки мыши - смены позиции муравья
def __placeAnt(event):
    global ant
    global pos
    x = c.canvasx(event.x)
    y = c.canvasy(event.y)
    tile_num = c.find_closest(x,y)[0]
    if tile_num == ant.id:
        return    
    ant.left()
    pos = ((tile_num-1)//TILES, tile_num%TILES-1)
    ant = tiles[pos[0]][pos[1]]
    ant.arrived()
    

# Обработка перемещения муравья на клетку
def move_ant(tile):
    global ant
    ant.left()
    ant = tile
    ant.arrived()    


# Сброс всего поля по кнопке и установка текущего паттерна движения
def reset():
    global ant
    global pos
    global head
    global step
    stop_ant()
    for line in tiles:
        for tile in line:
            tile.reset()
    ant = tiles[TILES//2][TILES//2]
    ant.arrived()
    head = 'N'
    step = cycle(pattern.get())
    pos = (TILES//2, TILES//2)


# Остановка муравья по кнопке
def stop_ant():
    global stop
    stop = True
    start_button.configure(text="Run", command=start)


# Старт муравья по кнопке
def start():
    global stop
    global pos
    global head
    stop = False
    start_button.configure(text="Pause", command=stop_ant)
    while not stop:
        move = MOVES[head][next(step)]
        head = move['head']
        pos = list(map(lambda x, y: x + y, pos, move['shift']))
        move_ant(tiles[pos[0]%TILES][pos[1]%TILES])
        root.update()
        sleep(.05)
        

# Выход из приложения по кнопке
def exit():
    global stop
    stop = True
    root.destroy()


# Привязка функций к кнопкам мыши
c.bind('<Button-1>', __switchTile)
c.bind('<Button-3>', __placeAnt)


# Кнопка сброса поля
reset_button = Button(root,
                      command=reset,
                      text='Set',
                      )
reset_button.place(relx=.4, rely=.95, anchor=CENTER)


# Окно ввода паттерна движения
pattern = StringVar()
pattern.set('RL')
pattern_entry = Entry(root,
                     textvariable=pattern,
                     )
pattern_entry.place(relx=.2, rely=.95, anchor=CENTER)


# Кнопка старта/стопа
start_button = Button(root,
                      command=start,
                      text='Run',
                      )
start_button.place(relx=.69, rely=.95, anchor=CENTER)


# Кнопка выхода
exit_button = Button(root,
                      command=exit,
                      text='Exit',
                      )
exit_button.place(relx=.9, rely=.95, anchor=CENTER)

    

# Начальные создание поля, помещение на него муравья и установка паттерна его движения
tiles = [[Tile(c, i*SIZE, j*SIZE, SIZE) for i in range(TILES)] for j in range(TILES)]
ant = tiles[TILES//2][TILES//2]
ant.arrived()
head = 'N'
step = cycle(pattern.get())
pos = (TILES//2, TILES//2)


# Запуск цикла ткинтера
root.mainloop()